In [ ]:
import numpy as np
from  netCDF4 import Dataset
import matplotlib.pyplot as plt

import time
#%matplotlib inline

In [ ]:
PATHbase="/mnt/web2/TsunamiSurge/JETS_System/JRC/"

In [ ]:
data0=Dataset(PATHbase+str(15610)+'/NETCDF_H.nc')

In [ ]:
nformat=data0.data_model

In [ ]:
ha0=data0.variables['HA'][:-1,:-1,:-1]
lon=data0.variables['LON'][:-1]
lat=data0.variables['LAT'][:-1]
t=data0.variables['TIME'][:-1]

latc=np.mean(lat)
lonc=np.mean(lon)

lons, lats = np.meshgrid(lon,lat)


In [ ]:
s=ha0.shape

In [ ]:
s

In [ ]:
s[1:]

In [ ]:
ens={}

In [ ]:
ens['nominal']=ha0[:,:,:]

In [ ]:
for f in range(2,4):
    data=Dataset(PATHbase+str(15610)+ '/%03i/coarse/NETCDF_H.nc' %(f))
    print f
    ha=data.variables['HA']
    print ha.shape
#    lon=data.variables['LON']
#    lat=data.variables['LAT']
#    t=data.variables['TIME']
    if ha.shape[1:]==s[1:] :
             ens['r'+str(f)]=ha[:,:,:]
    data.close()        


In [ ]:
tt = {k:np.shape(v)[0] for k,v in ens.items()}

In [ ]:
tm=np.min(tt.values())


In [ ]:
ss

In [ ]:
ss=np.append(tm,s[1:])
#w=np.zeros(ss)
wmin=ha0[:ss[0],:,:]
wmax=ha0[:ss[0],:,:]
for k,v in ens.items():
     w = w+v[:ss[0],:,:] 
     c=wmin>v[:ss[0],:,:]
     print np.sum(c) 
     wmin[c]=v[:ss[0],:,:][c]
     c=wmax<v[:ss[0],:,:]
     print np.sum(c) 
     wmax[c]=v[:ss[0],:,:][c]

    

In [ ]:
tm=200

In [ ]:
tstart = time.time()

In [ ]:
import copy

In [ ]:
wmin=copy.deepcopy(ha0)
wmax=copy.deepcopy(ha0)
#ss=np.append(tm,s[1:])
w=ha0

In [ ]:
ni=6

In [ ]:
for f in range(1,ni):
    data=Dataset(PATHbase+str(15610)+ '/%03i/coarse/NETCDF_H.nc' %(f))
    print f
    ha=data.variables['HA'][:-1,:-1,:-1]
    print ha.shape
#    lon=data.variables['LON']
#    lat=data.variables['LAT']
#    t=data.variables['TIME']
    c=wmin>ha
    print np.sum(c) 
    wmin[c]=ha[c]
    c=wmax<ha
    print np.sum(c) 
    wmax[c]=ha[c]

    w=w+ha
    #data.close()        


In [ ]:
ha.max()

In [ ]:
wmin.all()==wmax.all()

In [ ]:
print time.time() - tstart

In [ ]:
import os
import shutil

In [ ]:

for f in range(11,ni):
    ifile=PATHbase+str(15610)+ '/%03i/coarse/NETCDF_H.nc' %(f)
    shutil.copy(ifile, '/home/critechuser/brey/NETCDF_TEST/15610/f{}.nc'.format(f))

In [ ]:
1874/60.

In [ ]:
wmin.max(),wmin.min()

In [ ]:
wmax.max(),wmax.min()

In [ ]:
plt.figure()
plt.plot(wmin[:,320,40],'r-',ha0[:ss[0],320,40],'g-',wmax[:,320,40],'b-',w[:,320,40]/f,'k--')
plt.show()

In [ ]:
from mpl_toolkits.basemap import Basemap, shiftgrid
import matplotlib.animation as animation


In [ ]:

# make orthographic basemap.
m = Basemap(llcrnrlon=lon.min(),llcrnrlat=lat.min(),urcrnrlon=lon.max(),urcrnrlat=lat.max(),
            projection='cyl',lat_1=lat.min(),lat_2=lat.max(),lon_0=lonc,
            resolution ='l',area_thresh=1000.)
# create figure, add axes
fig1 = plt.figure(figsize=(8,10))
ax = fig1.add_axes([0.1,0.1,0.8,0.8])
# set desired contour levels.

# compute native x,y coordinates of grid.
x, y = m(lons, lats)
# define parallels and meridians to draw.
parallels = np.arange(-80.,90,20.)
meridians = np.arange(0.,360.,20.)
m.drawcoastlines(linewidth=1.5)
m.drawparallels(parallels)
m.drawmeridians(meridians)
# set plot title
ax.set_title('Watel level height')

# plot contours.
CS1 = m.contour(x,y,ha[100,:-1,:-1],linewidths=0.5,colors='k',animated=True)
CS2 = m.contourf(x,y,ha[100,:-1,:-1],cmap=plt.cm.jet,animated=True)
# add colorbar
cb = m.colorbar(CS2,"bottom", size="5%", pad="2%")
cb.set_label('ha')
m.fillcontinents(color = 'gray')
        
plt.show()

In [ ]:
lon[:-1]

In [ ]:
lat[:-1]

In [ ]:
def onclick(event):
    global ix, iy
    ix, iy = event.xdata, event.ydata
    print 'x = %d, y = %d'%(
        ix, iy)

    global coords
    coords.append((ix, iy))

#   if len(coords) == 2:
#       fig.canvas.mpl_disconnect(cid)

    return coords



In [ ]:
coords=[]
cid = fig1.canvas.mpl_connect('button_press_event', onclick)
print coords

# WRITE to NETCDF file

In [ ]:
import os
os.chdir('../../brey/NETCDF_TEST/')
!pwd

In [ ]:
wfile = 'output.nc'
ncfile = Dataset(wfile,'w',format=nformat)

#create dimensions
ncfile.createDimension('lat',lat.size)
ncfile.createDimension('lon',lon.size)
ncfile.createDimension('time',t.size)

#define variables
latitude = ncfile.createVariable('LAT','d',('lat',))
longitude = ncfile.createVariable('LON','d',('lon',))
ha = ncfile.createVariable('HA','d',('time','lat','lon'))
hmin = ncfile.createVariable('Hmin','d',('time','lat','lon'))
hmax = ncfile.createVariable('Hmax','d',('time','lat','lon'))
hav = ncfile.createVariable('Have','d',('time','lat','lon'))

longitude[:] = lon
latitude[:] = lat
ha[:] = ha0
time[:] = time
hmin[:]=wmin
hmax[:]=wmax
hav[:]=w

#close ncfile
ncfile.close()


In [ ]:
from netcdf_io import file_open,file_write, file_close

In [ ]:
handle=file_open('output.nc')

In [ ]:
s=file_write(handle,'LAT',lat)
s=file_write(handle,'LON',lon)
s=file_write(handle,'TIME',t)
s=file_write(handle,'HA',ha0[:,:,:])
s=file_write(handle,'Hmin',wmin)
s=file_write(handle,'Hmax',wmax)
s=file_write(handle,'Hav',w)

In [ ]:
file_close(handle)

In [ ]:
ha0

In [ ]:
read=Dataset('output.nc')

In [ ]:
read.variables

In [ ]:
h0=read.variables['HA']
#hmin=read.variables['Hmin']
#hmax=read.variables['Hmax']
#hav=read.variables['Hav']



In [ ]:
plt.figure()
plt.plot(hmin[:,320,400],'r-',label='min')
#plt.plot(h0[:,320,400],'g-',label='baseline')
plt.plot(hmax[:,320,400],'b-',label='max')
#plt.plot(hav[:,320,400]/52.,'k--',label='average')
plt.legend()
plt.show()

In [ ]:
h0.shape, ha0.shape

In [ ]:
h0[:,:,:].min(), ha0[:,:,:].min()

In [ ]:
h0[:,:,:].max(), ha0[:,:,:].max()

In [ ]:
hmin[:,320,400]==hav[:,320,400]